## Dados sobre a COVID-19 no Brasil

Repositório: https://github.com/wcota/covid19br/

Descrição dos dados: https://github.com/wcota/covid19br/blob/master/DESCRIPTION.md

In [ ]:
%config Completer.use_jedi = True

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

# import directly from GitHub
df = pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv")

# change date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# print all available columns
print(df.columns, end='\n')

df.sample(3)

In [ ]:
# filter data for Brazil, and show only the specified columns
data_BR = df.query("state == 'TOTAL'")[['date', 'state', 'newDeaths', 'deaths', 'deathsMS', 'newCases', 'totalCases', 'totalCasesMS', 'recovered', 'tests', 'vaccinated', 'vaccinated_second']]

# create new columns
data_BR['activeCases'] = data_BR['totalCases'] - data_BR['deaths'] - data_BR['recovered']
data_BR['activeCasesMS'] = data_BR['totalCasesMS'] - data_BR['deathsMS'] - data_BR['recovered']
data_BR['activeCasesDiff'] = data_BR['activeCases'] - data_BR['activeCasesMS']
data_BR['deathsDiff'] = data_BR['deaths'] - data_BR['deathsMS']
data_BR['newVaccinated'] = data_BR['vaccinated'].diff()
data_BR['newVaccinated_second'] = data_BR['vaccinated_second'].diff()

# display the dataframe
data_BR.sample(3)

In [ ]:
import cufflinks as cf
cf.go_offline()
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot
import plotly.express as px

#### Casos Ativos - Acumulado

In [ ]:
# data_BR.iplot(y=['activeCases', 'activeCasesMS'],
#               x='date',width=2.0,
#               secondary_y='activeCasesDiff', secondary_y_title='Diferença',
#               xTitle='Data', yTitle='Casos Ativos', title='Casos Ativos - MS x Consórcio')
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(
        x = data_BR['date'],
        y = data_BR['activeCases'],
        line=dict(width=3),
        name='Casos Ativos',
        text= data_BR['date'],
        hoverinfo='text',
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(
        x = data_BR['date'],
        y = data_BR['activeCasesMS'],
        line=dict(width=3),
        name='Casos Ativos - MS',
        text= data_BR['date'],
        hoverinfo='text',
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(
        x = data_BR['date'],
        y = data_BR['activeCasesDiff'],
        name='diferença',
        text= data_BR['date'],
        hoverinfo='text',
    ),
    secondary_y=True,
)

# customizações de layout
fig.update_layout(
    title='<b>Casos Ativos - MS x Consórcio</b>',
    xaxis_tickformat = '%d/%m/%Y',
    hovermode='x unified',
    separators=',.',
    plot_bgcolor='#fafafa',
    legend=dict(
        x=0.01,
        y=0.9,
        traceorder='normal',
        font=dict(size=10)
    )
)

fig.update_traces(hovertemplate='%{y:,.0f}')

# Anotações
fig.add_annotation(x=data_BR.loc[data_BR['activeCasesDiff'].idxmax(), 'date'], y=data_BR['activeCasesDiff'].max(),
            yref='y2',
            text="> "+str(round(data_BR['activeCasesDiff'].max(),-3)),
            font=dict(size=10),
            showarrow=True,
            arrowhead=1)

# Set x-axis title
# fig.update_xaxes(title_text="Data")

# Set y-axes titles
fig.update_yaxes(title_text="Casos Confirmados", secondary_y=False)
fig.update_yaxes(title_text="Diferença", secondary_y=True)

fig.show()
fig.write_html("graficos/leg-int/casos-ativos_x_consorcio.html")

#### Casos confirmados por Mês

In [ ]:
# agrupando por mês
per = data_BR['date'].dt.to_period('M')
p_mes = data_BR.groupby(per).agg({
    'newDeaths': 'sum', 
     'newCases': 'sum'}).to_timestamp()


In [ ]:
# Casos confirmados por mês

fig = go.Figure(
    data=[
        go.Bar(
            x = p_mes.index,
            y = p_mes['newCases'],            
            name='Casos',
        ),
    ]
)

fig.update_layout(
    title = '<b>Casos Confirmados por mês - COVID-19</b>',
    legend=dict(
        x=0.01,
        y=0.9,
        traceorder='normal',
        font=dict(size=10),
    ),
    hovermode='x unified', 
    separators=',.', 
    plot_bgcolor='#fafafa',
    xaxis_tickformat = '%m/%Y',
)
fig.update_xaxes(nticks=p_mes.shape[0])

fig.update_traces(
    hovertemplate='%{y:,.0f}',
    marker_color='goldenrod',    
)
fig.write_html("graficos/leg-int/casos-p-mes.html")
fig.show()

#### Óbitos - Acumulado

In [ ]:
# Óbitos acumulados

# data_BR.iplot(y=['deaths', 'deathsMS'],
#               x='date',width=2.0,
#               secondary_y='deathsDiff', secondary_y_title='Diferença',
#               xTitle='Data', yTitle='Óbitos', title='Óbitos - MS x Consórcio')

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(
        x = data_BR['date'],
        y = data_BR['deaths'],
        line=dict(width=3, color='orange'),
        name='Óbitos',
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(
        x = data_BR['date'],
        y = data_BR['deathsMS'],
        line=dict(width=3, color='blue'),
        name='Óbitos - MS',
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(
        x = data_BR['date'],
        y = data_BR['deathsDiff'],
        name='diferença',
    ),
    secondary_y=True,
)

# customização do layout
fig.update_layout(
    title='<b>Óbitos - MS x Consórcio</b>',
    xaxis_tickformat = '%d/%m/%Y',
    hovermode='x unified',
    separators=',.',
    plot_bgcolor='#fafafa',
    legend=dict(
        x=0.01,
        y=0.9,
        traceorder='normal',
        font=dict(size=10)),
)

fig.update_traces(hovertemplate='%{y:,.0f}')

# Anotações
fig.add_annotation(x=data_BR.loc[data_BR['deathsDiff'].idxmax(), 'date'], y=data_BR['deathsDiff'].max(),
            yref='y2',
            text="> "+str(round(data_BR['deathsDiff'].max(),-2)),
            font=dict(size=10),
            showarrow=True,
            arrowhead=1)

# Set x-axis title
# fig.update_xaxes(title_text="Data")

# Set y-axes titles
fig.update_yaxes(title_text="Óbitos Confirmados", secondary_y=False)
fig.update_yaxes(title_text="Diferença", secondary_y=True)

fig.show()
fig.write_html("graficos/leg-int/obitos_x_consorcio.html")

#### Óbitos registrados por Mês

In [ ]:
# Óbitos p/ mês

fig = go.Figure(
    data=[
        go.Bar(
            x = p_mes.index,
            y = p_mes['newDeaths'],            
            name='Óbitos',
        ),
    ]
)

fig.update_layout(
    title = '<b>Óbitos registrados por mês - COVID-19</b>',
    legend=dict(
        x=0.01,
        y=0.9,
        traceorder='normal',
        font=dict(size=10),
    ),
    hovermode='x unified', 
    separators=',.', 
    plot_bgcolor='#fafafa',
    xaxis_tickformat = '%m/%Y',
)
fig.update_xaxes(nticks=p_mes.shape[0])

fig.update_traces(
    hovertemplate='%{y:,.0f}',
    marker_color='firebrick',    
)
fig.write_html("graficos/leg-int/obitos-p-mes.html")
fig.show()

#### Indicadores - Vacinação

In [ ]:
import plotly.io as pio
pio.kaleido.scope.default_format = "svg"
pio.kaleido.scope.default_width = 200
pio.kaleido.scope.default_height = 200
pio.kaleido.scope.default_scale = 0.9

In [ ]:
# Indicador de Vacinas aplicadas

fig = go.Figure()

total_vacinados = data_BR['vaccinated'].iloc[-1] + data_BR['vaccinated_second'].iloc[-1]
# mean_7_vacinados = np.sum(data_BR[['vaccinated', 'vaccinated_second']].tail(7).values)/7

fig.add_trace(go.Indicator(
    mode = "number",
    value = total_vacinados,
    #delta = {'reference': mean_7_vacinados, 'relative': True},
    title= {'text': 'Doses aplicadas'},
    domain = {'x': [0.15, 0], 'y': [0.001, 0]}
    )
)
fig.write_image("graficos/indicadores/ind-qtd-vacinas.svg")
fig.show()

In [ ]:
# Indicador 1 Dose

fig = go.Figure()

total_vacinados = data_BR['vaccinated'].iloc[-1]
crescimento = data_BR['vaccinated'].iloc[-2]

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = total_vacinados,    
    number = {'valueformat': ',.4s',},
    delta = {'reference': crescimento, 'relative': False, 'valueformat': ',.5s'},
    title= {'text': '1ª Dose'},
    domain = {'x': [0.15, 0], 'y': [0.001, 0]}
    )
)
fig.write_image("graficos/indicadores/ind-qtd-1dose.svg")
fig.show()

In [ ]:
# Indicador 2 Dose

fig = go.Figure()

total_vacinados = data_BR['vaccinated_second'].iloc[-1]
crescimento = data_BR['vaccinated_second'].iloc[-2]

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = total_vacinados,
    number = {'valueformat': ',.4s',},
    delta = {'reference': crescimento, 'relative': False, 'valueformat': ',.5s'},
    title= {'text': '2ª Dose'},
    domain = {'x': [0.15, 0], 'y': [0.001, 0]}
    )
)
fig.write_image("graficos/indicadores/ind-qtd-2dose.svg")
fig.show()

In [ ]:
# Tempo decorrido desde o início da vacinação
t_menor = data_BR[~data_BR['vaccinated'].isna()]['date'].min()
t_maior = data_BR[~data_BR['vaccinated'].isna()]['date'].max()
t_decorrido = t_maior - t_menor
t_decorrido = int(str(t_decorrido).split()[0])
fig = go.Figure()
fig.add_trace(go.Indicator(
    mode = "number",
    value = t_decorrido,
    domain = {'x': [0.3, 0.3], 'y': [0.5, 1]},
    title= {'text': "Tempo decorrido desde<br>o início da vacinação<br><span style='font-size:0.8em;color:gray'>Em dias</span>"},
    )
)
fig.write_image("graficos/indicadores/ind-tempo-vacinacao.svg")
fig.show()

#### Evolução da Vacinação

In [ ]:
# Evolução da vacinação

# data_BR[~data_BR['vaccinated'].isna()].iplot(
#     x='date', 
#     y=['vaccinated', 'vaccinated_second'], 
#     title='Evolução da vacinação'    
# )

_dv = data_BR[~data_BR['vaccinated'].isna()]
fig = go.Figure(go.Scatter(
    x = _dv['date'],
    y = _dv['vaccinated'],
    line=dict(color='MediumPurple', width=3),
    fill='tonexty',
    name='primeira dose'

))

fig.add_trace(go.Scatter(
    x = _dv['date'],
    y = _dv['vaccinated_second'],
    line=dict(color='Coral', width=3),
    fill='tozeroy',
    name='segunda dose'
))

fig.update_layout(
    title = '<b>Evolução da vacinação - COVID-19</b>',
    xaxis_tickformat = '%d/%m/%Y',
    legend=dict(
        x=0.01,
        y=0.9,
        traceorder='normal',
        font=dict(size=10)),
)
fig.update_traces(hovertemplate='%{y:,.0f}')
fig.update_layout(hovermode='x unified', separators=',.', plot_bgcolor='#fafafa')
fig.show()
fig.write_html("graficos/leg-int/evolucao-vacinacao.html")

#### Vacinação Diária

In [ ]:
# vacinas aplicadas por dia

_dv['1_dose_7d'] = _dv['newVaccinated'].rolling(7).mean()
_dv['2_dose_7d'] = _dv['newVaccinated_second'].rolling(7).mean()

fig = go.Figure(
    data=[
        go.Bar(
            x = _dv['date'],
            y = _dv['newVaccinated'],            
            name='primeira dose',
            opacity=0.5
        ),
        go.Bar(
            x = _dv['date'],
            y = _dv['newVaccinated_second'],            
            name='segunda dose',
        ),
        go.Scatter(
            x = _dv['date'],
            y = _dv['1_dose_7d'],
            line=dict(color='MediumPurple', width=5),
            name='média móvel 1ª dose'
        ),
        go.Scatter(
            x = _dv['date'],
            y = _dv['2_dose_7d'],
            line=dict(color='Coral', width=5),
            name='média móvel 2ª dose'
        )
    ]
)

fig.update_layout(
    title = '<b>Vacinação por dia - COVID-19</b>',
    xaxis_tickformat = '%d/%m/%Y',
    barmode='stack',
    legend=dict(
        x=0.01,
        y=0.9,
        traceorder='normal',
        font=dict(size=10)),
)
fig.update_traces(
    hovertemplate='%{y:,.0f}',
    marker_color='DarkGray',    
)
fig.update_layout(hovermode='x unified', separators=',.', plot_bgcolor='#fafafa')
fig.show()
fig.write_html("graficos/leg-int/vacinacao-por-dia.html")


### Proporção da população vacinada por UF

Malha estadual - shp
Link: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/15774-malhas.html?=&t=downloads

População estimada - xls
Link: https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html?=&t=downloads

In [ ]:
import geopandas as gpd

geo_dados_uf = gpd.read_file('shapefiles/BR_UF_2020/BR_UF_2020.shp')
# geo_dados_uf = geo_dados_uf.to_crs('+proj=utm +zone=23 +south +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=km +no_defs ')
# geo_dados_uf.to_file("geojson/brasil-uf-compressed.json", driver='GeoJSON')
geo_dados_uf.head(3)

In [ ]:
pop_munic = pd.read_excel('datasets/originais/populacao_2020.xls', sheet_name='Municípios', 
                          skiprows=1, skipfooter=16)
pop_munic['POPULAÇÃO ESTIMADA'] = pop_munic['POPULAÇÃO ESTIMADA'].apply(lambda x: str(x).split('(')[0])
pop_munic['POPULAÇÃO ESTIMADA'] = pop_munic['POPULAÇÃO ESTIMADA'].astype(int)
pop_uf = pop_munic[['UF', 'POPULAÇÃO ESTIMADA']].groupby('UF').sum().reset_index()

In [ ]:
data_UF = df.query("state != 'TOTAL' and epi_week == @df['epi_week'].max()")

In [ ]:
data_UF['perc_vac'] = (data_UF['vaccinated'] / data_UF['state'].map(pop_uf.set_index('UF')['POPULAÇÃO ESTIMADA']))*100
data_UF['date_str'] = data_UF['date'].apply(lambda x: x.strftime('%d/%m/%Y'))

#### Usando GeoJson

In [ ]:
# Usando o GeoJson
import json

with open('geojson/brasil-uf-compressed.json') as f:
    uf_br = json.load(f)
    
uf_br["features"][0].keys()

In [ ]:
uf_br["features"][0]['properties'].keys()

In [ ]:
# Plotando mapa cloropletico usando GeoJson - com animação
# Mapa com a linha do tempo da última semana epidemiológica

# max_pct = _df_plot['perc_vac'].max()
fig = px.choropleth_mapbox(data_UF,
                           geojson=uf_br,
                           locations="state", 
                           featureidkey="properties.SIGLA_UF",
                           color='perc_vac',
                           color_continuous_scale=px.colors.sequential.Teal,
                           center={'lat': -16.701591, 'lon': -49.164524},
                           mapbox_style="carto-positron",
                           animation_frame="date_str",                                                     
                           opacity=0.6,
                           zoom=3)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [ ]:
# Plotando mapa cloropletico usando GeoJson - sem animação

_df_plot = data_UF.query("date == @df['date'].max()")

#------------------------------ Usando graph_objects e GeoJson
fig = go.Figure(go.Choroplethmapbox(
    geojson=uf_br,
    featureidkey="properties.SIGLA_UF",
    locations=_df_plot['state'], 
    z=_df_plot['perc_vac'],
    colorscale="Mint", 
    zmin=0, 
    zmax=30,
    marker_line_width=0.8,
    colorbar={'len': 0.97, 'nticks': 6, 'thickness': 15, 'borderwidth': 0, 'title': '% Vac.'},    
    hovertemplate='<b>%{properties.NM_UF}</b><br><br>Pop. Vacinada: %{z:.2f}%',
    name="",
    )
)
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=3, 
    mapbox_center = {'lat': -16.701591, 'lon': -49.164524},
    margin={"r":0,"t":0,"l":0,"b":0},
    title_x=0.7,
    title_y=0.96,
    separators=',.',
    title='<b>Vacinação por UF</b> - % da população vacinada',
)

fig.write_html("graficos/mapa_vacinacao.html")
fig.show()

#### Usando GeoDataframe

In [ ]:
# Plotando mapa cloropletico usando o express

_df_plot = data_UF.query("date == @df['date'].max()")
_df_plot = geo_dados_uf.merge(_df_plot, left_on='SIGLA_UF', right_on='state').set_index('SIGLA_UF')
_df_plot = _df_plot[['SIGLA_UF','NM_UF', 'perc_vac', 'geometry']].copy()

#------------------------------ Usando plotly express e GeoDataframe
fig = px.choropleth_mapbox(
    _df_plot, 
    geojson=_df_plot.geometry, 
    color="perc_vac",
    locations=_df_plot.index,
    color_continuous_scale=px.colors.sequential.Mint,
    center={'lat': -16.701591, 'lon': -49.164524},
    mapbox_style="open-street-map",
    title='<b>Vacinação por UF</b> - % da população vacinada',
    hover_name='NM_UF',
    hover_data={'perc_vac': ':,.2f %'},
    labels={'perc_vac': '% Vacinados',},
    zoom=3
    )
fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    title_x=0.7,
    title_y=0.96,
    separators=',.',
)
fig.update_geos(fitbounds="locations", visible=False)
fig.write_html("graficos/mapa_vacinacao.html")

fig.show()

### Nº de casos por Municípios - proporcional a 100.000 habitantes

In [ ]:
# import gzip file with cities data
chunk_size = 50000
df_cities = pd.DataFrame()

_dc = pd.read_csv("https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time.csv.gz?raw=true", 
                        compression='gzip',
                        iterator=True,
                        chunksize=chunk_size
                       )
for df in _dc:
    df_cities = df_cities.append(df)


df_cities.sample(3)

In [ ]:
df_cities['date'] = pd.to_datetime(df_cities['date'])

In [ ]:
df_cities['ibgeID'].dtype

In [ ]:
df_cities.shape

In [ ]:
df_gps_cities = pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/gps_cities.csv")
df_gps_cities.sample(3)

In [ ]:
df_gps_cities.shape

In [ ]:
# verifica se tem algum codigo ibge repetido
ibge_rep = np.any(df_gps_cities['ibgeID'].value_counts()>1)

# verifica se tem algum codigo fora do padrão
ibge_fora = bool(len(np.where(df_gps_cities['ibgeID'].values > 9999999)[0])) | bool(len(np.where(df_gps_cities['ibgeID'].values < 0)[0]))

# verifica se tem algum valor faltante
ibge_missing = df_gps_cities['ibgeID'].isna().sum()

print(f'cod. ibge repetido: {ibge_rep} \ncod. ibge fora do padrão: {ibge_fora} \nvalores faltantes: {ibge_missing}')

In [ ]:
# Verificando os registros cujo ibgeID está faltando
df_gps_cities.loc[df_gps_cities['ibgeID'].isna()]

In [ ]:
# removendo as linhas cujo campo ibgeID está faltando
df_gps_cities = df_gps_cities.dropna(subset=['ibgeID'])

In [ ]:
# convertendo o tipo da coluna ibeID do df_gps_cities para o mesmo tipo da coluna ibgeID do df_cities
df_gps_cities['ibgeID'] = df_gps_cities['ibgeID'].astype(int)

In [ ]:
df_gps_cities['ibgeID'].dtype

In [ ]:
# definindo as colunas 'lat' e 'lon' no df_cities com base no 'ibgeID' do df_gps_cities
df_cities['lat'] = df_cities['ibgeID'].map(df_gps_cities.set_index('ibgeID')['lat'])
df_cities['lon'] = df_cities['ibgeID'].map(df_gps_cities.set_index('ibgeID')['lon'])

In [ ]:
df_cities.sample(3)

In [ ]:
_df = df_cities.query('date == @df_cities.date.max()')
mapa = px.scatter_mapbox(
    _df, 
    lat='lat', 
    lon='lon',  
    hover_name='city',
    color_continuous_scale=px.colors.sequential.matter,
    color='totalCases_per_100k_inhabitants',                     
    zoom= 3,
    hover_data={'lat':False, 'lon':False, 'totalCases':True, 'deaths': True, 'totalCases_per_100k_inhabitants': ':.2f'},
    labels={'totalCases': 'Casos', 'deaths': 'Óbitos', 'totalCases_per_100k_inhabitants': 'Casos p/ 100mil hab.'},
    title='<b>Covid-19</b> Proporção de casos por 100mil habitantes'
)

mapa.update_layout(
    mapbox_style='open-street-map',
    height=600, 
    margin={'r':0, 't':0, 'l':0, 'b':0},
    title_x=0.5,
    title_y=0.96,
)
mapa.show()
mapa.write_html("graficos/mapa-casos-p-100k-h.html")

In [ ]:
import ipywidgets as widgets
from ipywidgets import fixed

In [ ]:
f_date = df_cities['date'].dt.strftime('%Y-%m-%d').sort_values().unique().tolist()

In [ ]:
# Controle Widget para data
date_limit = widgets.SelectionSlider(
    options= f_date,
    value= '2020-03-01',
    description= 'Data: ',
    continuous_update= False,
    orientation= 'horizontal',
    readout= True,
    style={'description_width': 'initial'}
)

def update_map(df, limit):


    _df = df.loc[df['date'] == pd.to_datetime(limit)].copy()
#     print(limit, end='\n')
#     print(pd.to_datetime(limit), end='\n')
#     print(_df, end='\n')
    mapa = px.scatter_mapbox( _df, lat='lat', lon='lon',  
                     hover_name='city',
                     hover_data=['totalCases', 'deaths'],
                     color_continuous_scale=px.colors.sequential.Plasma_r,
                     color='totalCases_per_100k_inhabitants',                                          
                     zoom= 3)
    mapa.update_layout(mapbox_style='open-street-map')
    mapa.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
    mapa.show()

In [ ]:
# _df = df_cities.loc[df_cities['date'] == pd.to_datetime('2020-03-01')]

widgets.interactive( update_map, df=fixed(df_cities), limit=date_limit)